In [1]:
# RUN
import sys
sys.path.append("/opt/src")
import mip_functions as mip
import calculate_prevalences as cap
import probe_summary_generator
import pickle
import json
import copy
import math
import os
import numpy as np
import subprocess
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from matplotlib.lines import Line2D
plt.rcParams['svg.fonttype'] = 'none'
import pandas as pd
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
import allel
wdir = "/opt/analysis/"
data_dir = "/opt/data/"
import plotly.express as px
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from urllib.request import urlopen

Classes reloading.
functions reloading


# Calling genotypes, prevalences and filtering data
The original vcf file created by freebayes contain the genotypes determined by the program itself. In addition, genotype values for aggregated and non-aggregated nucleotides and aminoacids are also available as *_genotypes_table.csv files as described above.  

However, the default parameters generating the vcf file are not very strict. In this part of the analysis we will apply various filters to the count tables and generate genotype calls based on those filters.

### Chose which tables to analyse
Select the type of data to analyse. Make sure the count file is matching the coverage file. e.g. alternate_XX_table and coverage_XX_table, XX must be the same value (AA, AN or nothing).

#### Example cell
```python
mutation_count_file = "/opt/analysis/alternate_AA_table.csv"
mutation_coverage_file = "/opt/analysis/coverage_AA_table.csv"
```

In [2]:
# USER INPUT

mutation_count_file = "alternate_AA_table.csv"
mutation_coverage_file = "coverage_AA_table.csv"

In [3]:
# RUN
import pandas as pd
mutation_counts = pd.read_csv(mutation_count_file,
                              header=list(range(6)),
                              index_col=0)
mutation_counts.head()

Gene ID              PF3D7_0417200                                    \
Gene                       dhfr-ts                                     
Mutation Name     dhfr-ts-Ala16Val dhfr-ts-Asn51Ile dhfr-ts-Cys59Arg   
ExonicFunc        missense_variant missense_variant missense_variant   
AA Change                 Ala16Val         Asn51Ile         Cys59Arg   
Targeted                       Yes              Yes              Yes   
KB-04-06-PRX-04-1              0.0              2.0              2.0   
AG-00-45-PRX-00-1              0.0             30.0             30.0   
KB-07-13-PRX-07-1              0.0            254.0            254.0   
AM-04-08-PRX-04-1              0.0             18.0             18.0   
AM-07-84-PRX-07-1              0.0             51.0             51.0   

Gene ID                                                                  \
Gene                                                                      
Mutation Name     dhfr-ts-Ile164Leu dhfr-ts-Ser108Asn dhfr-ts-Ser108Thr   
ExonicFunc         missense_variant  missense_variant  missense_variant   
AA Change                 Ile164Leu         Ser108Asn         Ser108Thr   
Targeted                        Yes               Yes               Yes   
KB-04-06-PRX-04-1               7.0               6.0               0.0   
AG-00-45-PRX-00-1               0.0              11.0               0.0   
KB-07-13-PRX-07-1              87.0             144.0               0.0   
AM-04-08-PRX-04-1               2.0              10.0               0.0   
AM-07-84-PRX-07-1              25.0              36.0               0.0   

Gene ID              PF3D7_0523000                   \
Gene                          mdr1                    
Mutation Name       mdr1-Ala821Thr  mdr1-Asn1042Asp   
ExonicFunc        missense_variant missense_variant   
AA Change                Ala821Thr       Asn1042Asp   
Targeted                        No              Yes   
KB-04-06-PRX-04-1              0.0              0.0   
AG-00-45-PRX-00-1              0.0              0.0   
KB-07-13-PRX-07-1              0.0              0.0   
AM-04-08-PRX-04-1              0.0              0.0   
AM-07-84-PRX-07-1              0.0              0.0   

Gene ID                                                             \
Gene                                                                 
Mutation Name     mdr1-Asn644_Asn654delinsLysAsnAsnAsnAsnAsnAspAsp   
ExonicFunc          missense_variant&conservative_inframe_deletion   
AA Change              Asn644_Asn654delinsLysAsnAsnAsnAsnAsnAspAsp   
Targeted                                                        No   
KB-04-06-PRX-04-1                                              0.0   
AG-00-45-PRX-00-1                                              0.0   
KB-07-13-PRX-07-1                                              0.0   
AM-04-08-PRX-04-1                                              0.0   
AM-07-84-PRX-07-1                                              0.0   

Gene ID                                                           ...  \
Gene                                                              ...   
Mutation Name                     mdr1-Asn647_Asn654delinsAspAsp  ...   
ExonicFunc        missense_variant&conservative_inframe_deletion  ...   
AA Change                              Asn647_Asn654delinsAspAsp  ...   
Targeted                                                      No  ...   
KB-04-06-PRX-04-1                                            0.0  ...   
AG-00-45-PRX-00-1                                            0.0  ...   
KB-07-13-PRX-07-1                                            0.0  ...   
AM-04-08-PRX-04-1                                            0.0  ...   
AM-07-84-PRX-07-1                                            0.0  ...   

Gene ID              PF3D7_1408000                                    \
Gene                           PM2                                     
Mutation Name        PM2-Asp440Asn    PM2-Gln442His    P

In [4]:
# RUN
mutation_coverage = pd.read_csv(mutation_coverage_file,
                                index_col=0,
                                header=list(range(6)))
mutation_coverage.head()

Gene ID              PF3D7_0417200                                    \
Gene                       dhfr-ts                                     
Mutation Name     dhfr-ts-Ala16Val dhfr-ts-Asn51Ile dhfr-ts-Cys59Arg   
ExonicFunc        missense_variant missense_variant missense_variant   
AA Change                 Ala16Val         Asn51Ile         Cys59Arg   
Targeted                       Yes              Yes              Yes   
KB-04-06-PRX-04-1              2.0              2.0              2.0   
AG-00-45-PRX-00-1             24.0             30.0             30.0   
KB-07-13-PRX-07-1            179.0            254.0            254.0   
AM-04-08-PRX-04-1             17.0             18.0             18.0   
AM-07-84-PRX-07-1             34.0             51.0             51.0   

Gene ID                                                                  \
Gene                                                                      
Mutation Name     dhfr-ts-Ile164Leu dhfr-ts-Ser108Asn dhfr-ts-Ser108Thr   
ExonicFunc         missense_variant  missense_variant  missense_variant   
AA Change                 Ile164Leu         Ser108Asn         Ser108Thr   
Targeted                        Yes               Yes               Yes   
KB-04-06-PRX-04-1               7.0               6.0               6.0   
AG-00-45-PRX-00-1               6.0              11.0              11.0   
KB-07-13-PRX-07-1              87.0             144.0             144.0   
AM-04-08-PRX-04-1              11.0              10.0              10.0   
AM-07-84-PRX-07-1              25.0              36.0              36.0   

Gene ID              PF3D7_0523000                   \
Gene                          mdr1                    
Mutation Name       mdr1-Ala821Thr  mdr1-Asn1042Asp   
ExonicFunc        missense_variant missense_variant   
AA Change                Ala821Thr       Asn1042Asp   
Targeted                        No              Yes   
KB-04-06-PRX-04-1              4.0              9.0   
AG-00-45-PRX-00-1              4.0            103.0   
KB-07-13-PRX-07-1            131.0            759.0   
AM-04-08-PRX-04-1             15.0             45.0   
AM-07-84-PRX-07-1             26.0            106.0   

Gene ID                                                             \
Gene                                                                 
Mutation Name     mdr1-Asn644_Asn654delinsLysAsnAsnAsnAsnAsnAspAsp   
ExonicFunc          missense_variant&conservative_inframe_deletion   
AA Change              Asn644_Asn654delinsLysAsnAsnAsnAsnAsnAspAsp   
Targeted                                                        No   
KB-04-06-PRX-04-1                                              1.0   
AG-00-45-PRX-00-1                                             11.0   
KB-07-13-PRX-07-1                                            221.0   
AM-04-08-PRX-04-1                                              3.0   
AM-07-84-PRX-07-1                                             37.0   

Gene ID                                                           ...  \
Gene                                                              ...   
Mutation Name                     mdr1-Asn647_Asn654delinsAspAsp  ...   
ExonicFunc        missense_variant&conservative_inframe_deletion  ...   
AA Change                              Asn647_Asn654delinsAspAsp  ...   
Targeted                                                      No  ...   
KB-04-06-PRX-04-1                                            1.0  ...   
AG-00-45-PRX-00-1                                           11.0  ...   
KB-07-13-PRX-07-1                                          221.0  ...   
AM-04-08-PRX-04-1                                            3.0  ...   
AM-07-84-PRX-07-1                                           37.0  ...   

Gene ID              PF3D7_1408000                                    \
Gene                           PM2                                     
Mutation Name        PM2-Asp440Asn    PM2-Gln442His    P

### Set your filters   
1.  **min_coverage**: how many UMIs are needed to for a genomic position for a sample to reliably call genotypes. If we set min_coverage = 10, any locus within a sample that is covered below this threshold will have an NA genotype.
2.  **min_count**: if a genomic position have enough coverage, how many UMIs supporting an ALT allele call is needed for a reliable call. If we set min_count = 2, any mutation with an  call that has less than 2 UMIs supporting the ALT call will revert to REF.
3.  **min_freq**: a minimum within sample allele frequency threshold to consider a variant valid. If set to 0.01, for example, a variant locus in a sample that is at 0.005 frequency for the ALT allele within the sample, the locus would be called REF, if the within sample AF is between 0.01 and 0.99, it would be considered HET, and if > 0.99, it would be homozygous ALT.

#### Example cell
```python
# filter mutation counts for minimum count parameter
# by setting counts to zero if it is below threshold
min_count = 2
# filter loci without enough coverage by setting
# coverage to zero if it is below threshold
min_coverage = 10
# call genotypes using the minimum within sample
# allele frequency
min_freq = 0
```

In [5]:
# USER INPUT 

# filter mutation counts for minimum count parameter
# by setting counts to zero if it is below threshold
min_count = 2
# filter loci without enough coverage by setting
# coverage to zero if it is below threshold
min_coverage = 10
# call genotypes using the minimum within sample
# allele frequency
min_freq = 0

In [6]:
# RUN

# import the PCA module which has genotype calling and
# filtering functions 
import PCA

gt_calls = PCA.call_genotypes(mutation_counts, mutation_coverage,
                              min_count, min_coverage, min_freq)
gt_calls.keys()

dict_keys(['genotypes', 'prevalences', 'wsaf', 'filtered_mutation_counts', 'filtered_mutation_coverage'])

### What are the dataframes generated by call_genotypes function and how  are they generated?

**filtered_mutation_counts**: take the mutation_counts table, if a cell's value is below *min_count*, reset that cell's value to zero, otherwise leave as is.  

In [7]:
# RUN
filtered_mutation_counts = gt_calls["filtered_mutation_counts"]
filtered_mutation_counts.to_csv(os.path.join(
        wdir, "filtered_alternate_AA_table.csv"))
filtered_mutation_counts.head()

Gene ID              PF3D7_0417200                                    \
Gene                       dhfr-ts                                     
Mutation Name     dhfr-ts-Ala16Val dhfr-ts-Asn51Ile dhfr-ts-Cys59Arg   
ExonicFunc        missense_variant missense_variant missense_variant   
AA Change                 Ala16Val         Asn51Ile         Cys59Arg   
Targeted                       Yes              Yes              Yes   
KB-04-06-PRX-04-1                0              2.0              2.0   
AG-00-45-PRX-00-1                0             30.0             30.0   
KB-07-13-PRX-07-1                0            254.0            254.0   
AM-04-08-PRX-04-1                0             18.0             18.0   
AM-07-84-PRX-07-1                0             51.0             51.0   

Gene ID                                                                  \
Gene                                                                      
Mutation Name     dhfr-ts-Ile164Leu dhfr-ts-Ser108Asn dhfr-ts-Ser108Thr   
ExonicFunc         missense_variant  missense_variant  missense_variant   
AA Change                 Ile164Leu         Ser108Asn         Ser108Thr   
Targeted                        Yes               Yes               Yes   
KB-04-06-PRX-04-1               7.0               6.0                 0   
AG-00-45-PRX-00-1               0.0              11.0                 0   
KB-07-13-PRX-07-1              87.0             144.0                 0   
AM-04-08-PRX-04-1               2.0              10.0                 0   
AM-07-84-PRX-07-1              25.0              36.0                 0   

Gene ID              PF3D7_0523000                   \
Gene                          mdr1                    
Mutation Name       mdr1-Ala821Thr  mdr1-Asn1042Asp   
ExonicFunc        missense_variant missense_variant   
AA Change                Ala821Thr       Asn1042Asp   
Targeted                        No              Yes   
KB-04-06-PRX-04-1              0.0                0   
AG-00-45-PRX-00-1              0.0                0   
KB-07-13-PRX-07-1              0.0                0   
AM-04-08-PRX-04-1              0.0                0   
AM-07-84-PRX-07-1              0.0                0   

Gene ID                                                             \
Gene                                                                 
Mutation Name     mdr1-Asn644_Asn654delinsLysAsnAsnAsnAsnAsnAspAsp   
ExonicFunc          missense_variant&conservative_inframe_deletion   
AA Change              Asn644_Asn654delinsLysAsnAsnAsnAsnAsnAspAsp   
Targeted                                                        No   
KB-04-06-PRX-04-1                                              0.0   
AG-00-45-PRX-00-1                                              0.0   
KB-07-13-PRX-07-1                                              0.0   
AM-04-08-PRX-04-1                                              0.0   
AM-07-84-PRX-07-1                                              0.0   

Gene ID                                                           ...  \
Gene                                                              ...   
Mutation Name                     mdr1-Asn647_Asn654delinsAspAsp  ...   
ExonicFunc        missense_variant&conservative_inframe_deletion  ...   
AA Change                              Asn647_Asn654delinsAspAsp  ...   
Targeted                                                      No  ...   
KB-04-06-PRX-04-1                                            0.0  ...   
AG-00-45-PRX-00-1                                            0.0  ...   
KB-07-13-PRX-07-1                                            0.0  ...   
AM-04-08-PRX-04-1                                            0.0  ...   
AM-07-84-PRX-07-1                                            0.0  ...   

Gene ID              PF3D7_1408000                                    \
Gene                           PM2                                     
Mutation Name        PM2-Asp440Asn    PM2-Gln442His    P

**filtered_mutation_coverage**: take the mutation_coverage table, if a cell's value is below *min_coverage*, reset that cell's value to zero, otherwise leave as is.

In [8]:
# RUN
filtered_mutation_coverage = gt_calls["filtered_mutation_coverage"]
filtered_mutation_coverage.to_csv(os.path.join(
        wdir, "filtered_coverage_AA_table.csv"))
filtered_mutation_coverage.head()

Gene ID              PF3D7_0417200                                    \
Gene                       dhfr-ts                                     
Mutation Name     dhfr-ts-Ala16Val dhfr-ts-Asn51Ile dhfr-ts-Cys59Arg   
ExonicFunc        missense_variant missense_variant missense_variant   
AA Change                 Ala16Val         Asn51Ile         Cys59Arg   
Targeted                       Yes              Yes              Yes   
KB-04-06-PRX-04-1              0.0              0.0              0.0   
AG-00-45-PRX-00-1             24.0             30.0             30.0   
KB-07-13-PRX-07-1            179.0            254.0            254.0   
AM-04-08-PRX-04-1             17.0             18.0             18.0   
AM-07-84-PRX-07-1             34.0             51.0             51.0   

Gene ID                                                                  \
Gene                                                                      
Mutation Name     dhfr-ts-Ile164Leu dhfr-ts-Ser108Asn dhfr-ts-Ser108Thr   
ExonicFunc         missense_variant  missense_variant  missense_variant   
AA Change                 Ile164Leu         Ser108Asn         Ser108Thr   
Targeted                        Yes               Yes               Yes   
KB-04-06-PRX-04-1               0.0               0.0               0.0   
AG-00-45-PRX-00-1               0.0              11.0              11.0   
KB-07-13-PRX-07-1              87.0             144.0             144.0   
AM-04-08-PRX-04-1              11.0              10.0              10.0   
AM-07-84-PRX-07-1              25.0              36.0              36.0   

Gene ID              PF3D7_0523000                   \
Gene                          mdr1                    
Mutation Name       mdr1-Ala821Thr  mdr1-Asn1042Asp   
ExonicFunc        missense_variant missense_variant   
AA Change                Ala821Thr       Asn1042Asp   
Targeted                        No              Yes   
KB-04-06-PRX-04-1              0.0              0.0   
AG-00-45-PRX-00-1              0.0            103.0   
KB-07-13-PRX-07-1            131.0            759.0   
AM-04-08-PRX-04-1             15.0             45.0   
AM-07-84-PRX-07-1             26.0            106.0   

Gene ID                                                             \
Gene                                                                 
Mutation Name     mdr1-Asn644_Asn654delinsLysAsnAsnAsnAsnAsnAspAsp   
ExonicFunc          missense_variant&conservative_inframe_deletion   
AA Change              Asn644_Asn654delinsLysAsnAsnAsnAsnAsnAspAsp   
Targeted                                                        No   
KB-04-06-PRX-04-1                                              0.0   
AG-00-45-PRX-00-1                                             11.0   
KB-07-13-PRX-07-1                                            221.0   
AM-04-08-PRX-04-1                                              0.0   
AM-07-84-PRX-07-1                                             37.0   

Gene ID                                                           ...  \
Gene                                                              ...   
Mutation Name                     mdr1-Asn647_Asn654delinsAspAsp  ...   
ExonicFunc        missense_variant&conservative_inframe_deletion  ...   
AA Change                              Asn647_Asn654delinsAspAsp  ...   
Targeted                                                      No  ...   
KB-04-06-PRX-04-1                                            0.0  ...   
AG-00-45-PRX-00-1                                           11.0  ...   
KB-07-13-PRX-07-1                                          221.0  ...   
AM-04-08-PRX-04-1                                            0.0  ...   
AM-07-84-PRX-07-1                                           37.0  ...   

Gene ID              PF3D7_1408000                                    \
Gene                           PM2                                     
Mutation Name        PM2-Asp440Asn    PM2-Gln442His    P

**wsaf**: divide *filtered_mutation_counts* table by *filtered_mutation_coverage* table, yielding within sample allele frequencies.  

In [9]:
# RUN
freq = gt_calls["wsaf"]
freq.to_csv(os.path.join(
        wdir, "within_sample_allele_frequencies.csv"))
freq.head()

Gene ID              PF3D7_0417200                                    \
Gene                       dhfr-ts                                     
Mutation Name     dhfr-ts-Ala16Val dhfr-ts-Asn51Ile dhfr-ts-Cys59Arg   
ExonicFunc        missense_variant missense_variant missense_variant   
AA Change                 Ala16Val         Asn51Ile         Cys59Arg   
Targeted                       Yes              Yes              Yes   
KB-04-06-PRX-04-1              NaN              NaN              NaN   
AG-00-45-PRX-00-1              0.0              1.0              1.0   
KB-07-13-PRX-07-1              0.0              1.0              1.0   
AM-04-08-PRX-04-1              0.0              1.0              1.0   
AM-07-84-PRX-07-1              0.0              1.0              1.0   

Gene ID                                                                  \
Gene                                                                      
Mutation Name     dhfr-ts-Ile164Leu dhfr-ts-Ser108Asn dhfr-ts-Ser108Thr   
ExonicFunc         missense_variant  missense_variant  missense_variant   
AA Change                 Ile164Leu         Ser108Asn         Ser108Thr   
Targeted                        Yes               Yes               Yes   
KB-04-06-PRX-04-1               NaN               NaN               NaN   
AG-00-45-PRX-00-1               NaN               1.0               0.0   
KB-07-13-PRX-07-1          1.000000               1.0               0.0   
AM-04-08-PRX-04-1          0.181818               1.0               0.0   
AM-07-84-PRX-07-1          1.000000               1.0               0.0   

Gene ID              PF3D7_0523000                   \
Gene                          mdr1                    
Mutation Name       mdr1-Ala821Thr  mdr1-Asn1042Asp   
ExonicFunc        missense_variant missense_variant   
AA Change                Ala821Thr       Asn1042Asp   
Targeted                        No              Yes   
KB-04-06-PRX-04-1              NaN              NaN   
AG-00-45-PRX-00-1              NaN              0.0   
KB-07-13-PRX-07-1              0.0              0.0   
AM-04-08-PRX-04-1              0.0              0.0   
AM-07-84-PRX-07-1              0.0              0.0   

Gene ID                                                             \
Gene                                                                 
Mutation Name     mdr1-Asn644_Asn654delinsLysAsnAsnAsnAsnAsnAspAsp   
ExonicFunc          missense_variant&conservative_inframe_deletion   
AA Change              Asn644_Asn654delinsLysAsnAsnAsnAsnAsnAspAsp   
Targeted                                                        No   
KB-04-06-PRX-04-1                                              NaN   
AG-00-45-PRX-00-1                                              0.0   
KB-07-13-PRX-07-1                                              0.0   
AM-04-08-PRX-04-1                                              NaN   
AM-07-84-PRX-07-1                                              0.0   

Gene ID                                                           ...  \
Gene                                                              ...   
Mutation Name                     mdr1-Asn647_Asn654delinsAspAsp  ...   
ExonicFunc        missense_variant&conservative_inframe_deletion  ...   
AA Change                              Asn647_Asn654delinsAspAsp  ...   
Targeted                                                      No  ...   
KB-04-06-PRX-04-1                                            NaN  ...   
AG-00-45-PRX-00-1                                            0.0  ...   
KB-07-13-PRX-07-1                                            0.0  ...   
AM-04-08-PRX-04-1                                            NaN  ...   
AM-07-84-PRX-07-1                                            0.0  ...   

Gene ID              PF3D7_1408000                                    \
Gene                           PM2                                     
Mutation Name        PM2-Asp440Asn    PM2-Gln442His    P

**genotypes**: take the *wsaf* table, if a cell's value is less than *min_freq* set the genotype value to 0 (homozygous wild type); if the cell's value is more than (*1 - min_freq*) set the genotype value to 2 (homozygous mutant), if the cell's value is between *min_freq* and (*1 - min_freq*) set the genotype value to 1 (heterozygous/mixed).  

In [10]:
# RUN
genotypes = gt_calls["genotypes"]
genotypes.to_csv(os.path.join(
        wdir, "filtered_genotypes_table.csv"))
genotypes.head()

Gene ID              PF3D7_0417200                                    \
Gene                       dhfr-ts                                     
Mutation Name     dhfr-ts-Ala16Val dhfr-ts-Asn51Ile dhfr-ts-Cys59Arg   
ExonicFunc        missense_variant missense_variant missense_variant   
AA Change                 Ala16Val         Asn51Ile         Cys59Arg   
Targeted                       Yes              Yes              Yes   
KB-04-06-PRX-04-1              NaN              NaN              NaN   
AG-00-45-PRX-00-1              0.0              2.0              2.0   
KB-07-13-PRX-07-1              0.0              2.0              2.0   
AM-04-08-PRX-04-1              0.0              2.0              2.0   
AM-07-84-PRX-07-1              0.0              2.0              2.0   

Gene ID                                                                  \
Gene                                                                      
Mutation Name     dhfr-ts-Ile164Leu dhfr-ts-Ser108Asn dhfr-ts-Ser108Thr   
ExonicFunc         missense_variant  missense_variant  missense_variant   
AA Change                 Ile164Leu         Ser108Asn         Ser108Thr   
Targeted                        Yes               Yes               Yes   
KB-04-06-PRX-04-1               NaN               NaN               NaN   
AG-00-45-PRX-00-1               NaN               2.0               0.0   
KB-07-13-PRX-07-1               2.0               2.0               0.0   
AM-04-08-PRX-04-1               1.0               2.0               0.0   
AM-07-84-PRX-07-1               2.0               2.0               0.0   

Gene ID              PF3D7_0523000                   \
Gene                          mdr1                    
Mutation Name       mdr1-Ala821Thr  mdr1-Asn1042Asp   
ExonicFunc        missense_variant missense_variant   
AA Change                Ala821Thr       Asn1042Asp   
Targeted                        No              Yes   
KB-04-06-PRX-04-1              NaN              NaN   
AG-00-45-PRX-00-1              NaN              0.0   
KB-07-13-PRX-07-1              0.0              0.0   
AM-04-08-PRX-04-1              0.0              0.0   
AM-07-84-PRX-07-1              0.0              0.0   

Gene ID                                                             \
Gene                                                                 
Mutation Name     mdr1-Asn644_Asn654delinsLysAsnAsnAsnAsnAsnAspAsp   
ExonicFunc          missense_variant&conservative_inframe_deletion   
AA Change              Asn644_Asn654delinsLysAsnAsnAsnAsnAsnAspAsp   
Targeted                                                        No   
KB-04-06-PRX-04-1                                              NaN   
AG-00-45-PRX-00-1                                              0.0   
KB-07-13-PRX-07-1                                              0.0   
AM-04-08-PRX-04-1                                              NaN   
AM-07-84-PRX-07-1                                              0.0   

Gene ID                                                           ...  \
Gene                                                              ...   
Mutation Name                     mdr1-Asn647_Asn654delinsAspAsp  ...   
ExonicFunc        missense_variant&conservative_inframe_deletion  ...   
AA Change                              Asn647_Asn654delinsAspAsp  ...   
Targeted                                                      No  ...   
KB-04-06-PRX-04-1                                            NaN  ...   
AG-00-45-PRX-00-1                                            0.0  ...   
KB-07-13-PRX-07-1                                            0.0  ...   
AM-04-08-PRX-04-1                                            NaN  ...   
AM-07-84-PRX-07-1                                            0.0  ...   

Gene ID              PF3D7_1408000                                    \
Gene                           PM2                                     
Mutation Name        PM2-Asp440Asn    PM2-Gln442His    P

**prevalences**: take the *genotypes* table, if a cell's value is 2, reset its value to 1; otherwise leave as is.

In [11]:
# RUN
prevalences = gt_calls["prevalences"]
prevalences.to_csv(os.path.join(wdir, "prevalences_input_table.csv"))
prevalences.head()

Gene ID              PF3D7_0417200                                    \
Gene                       dhfr-ts                                     
Mutation Name     dhfr-ts-Ala16Val dhfr-ts-Asn51Ile dhfr-ts-Cys59Arg   
ExonicFunc        missense_variant missense_variant missense_variant   
AA Change                 Ala16Val         Asn51Ile         Cys59Arg   
Targeted                       Yes              Yes              Yes   
KB-04-06-PRX-04-1              NaN              NaN              NaN   
AG-00-45-PRX-00-1              0.0              1.0              1.0   
KB-07-13-PRX-07-1              0.0              1.0              1.0   
AM-04-08-PRX-04-1              0.0              1.0              1.0   
AM-07-84-PRX-07-1              0.0              1.0              1.0   

Gene ID                                                                  \
Gene                                                                      
Mutation Name     dhfr-ts-Ile164Leu dhfr-ts-Ser108Asn dhfr-ts-Ser108Thr   
ExonicFunc         missense_variant  missense_variant  missense_variant   
AA Change                 Ile164Leu         Ser108Asn         Ser108Thr   
Targeted                        Yes               Yes               Yes   
KB-04-06-PRX-04-1               NaN               NaN               NaN   
AG-00-45-PRX-00-1               NaN               1.0               0.0   
KB-07-13-PRX-07-1               1.0               1.0               0.0   
AM-04-08-PRX-04-1               1.0               1.0               0.0   
AM-07-84-PRX-07-1               1.0               1.0               0.0   

Gene ID              PF3D7_0523000                   \
Gene                          mdr1                    
Mutation Name       mdr1-Ala821Thr  mdr1-Asn1042Asp   
ExonicFunc        missense_variant missense_variant   
AA Change                Ala821Thr       Asn1042Asp   
Targeted                        No              Yes   
KB-04-06-PRX-04-1              NaN              NaN   
AG-00-45-PRX-00-1              NaN              0.0   
KB-07-13-PRX-07-1              0.0              0.0   
AM-04-08-PRX-04-1              0.0              0.0   
AM-07-84-PRX-07-1              0.0              0.0   

Gene ID                                                             \
Gene                                                                 
Mutation Name     mdr1-Asn644_Asn654delinsLysAsnAsnAsnAsnAsnAspAsp   
ExonicFunc          missense_variant&conservative_inframe_deletion   
AA Change              Asn644_Asn654delinsLysAsnAsnAsnAsnAsnAspAsp   
Targeted                                                        No   
KB-04-06-PRX-04-1                                              NaN   
AG-00-45-PRX-00-1                                              0.0   
KB-07-13-PRX-07-1                                              0.0   
AM-04-08-PRX-04-1                                              NaN   
AM-07-84-PRX-07-1                                              0.0   

Gene ID                                                           ...  \
Gene                                                              ...   
Mutation Name                     mdr1-Asn647_Asn654delinsAspAsp  ...   
ExonicFunc        missense_variant&conservative_inframe_deletion  ...   
AA Change                              Asn647_Asn654delinsAspAsp  ...   
Targeted                                                      No  ...   
KB-04-06-PRX-04-1                                            NaN  ...   
AG-00-45-PRX-00-1                                            0.0  ...   
KB-07-13-PRX-07-1                                            0.0  ...   
AM-04-08-PRX-04-1                                            NaN  ...   
AM-07-84-PRX-07-1                                            0.0  ...   

Gene ID              PF3D7_1408000                                    \
Gene                           PM2                                     
Mutation Name        PM2-Asp440Asn    PM2-Gln442His    P

## Filter genotypes / prevalences
It is generally a good idea to do some basic noise removal once the genotypes are created. Some suggestions are provided here.

### Filter variants that are always at low WSAF
If a variant is only seen at a low frequency within samples, it is a good indication that it could be just noise. Here we will set a number of samples and minimum WSAF threshold to remove such noise.

```python
num_samples_wsaf = 2
min_wsaf = 0.5
wsaf_filter = ((freq > min_wsaf).sum()) >= num_samples_wsaf
```

The above options will keep the variants that are in at > 0.5 WSAF in at least 2 samples.

In [12]:
# USER INPUT
num_samples_wsaf = 2
min_wsaf = 0.5

In [13]:
wsaf_filter = ((freq > min_wsaf).sum()) >= num_samples_wsaf
print(("{} of {} variants will remain after the wsaf filter").format(
    wsaf_filter.sum(), freq.shape[1]))

34 of 133 variants will remain after the wsaf filter


### Filter variants that are observed with low UMI counts
If a variant is only supported by a low number of UMIs across the entire sample set, it is another indication of noise.

```python
num_samples_umi = 2
min_umi = 3
umi_filter = ((filtered_mutation_counts >= min_umi).sum()) > num_samples_umi
```

The above options will keep the variants that are supported by at least 3 UMIs in at least 2 samples.

In [14]:
# USER INPUT
num_samples_umi = 2
min_umi = 3

In [15]:
# RUN
umi_filter = ((filtered_mutation_counts >= min_umi).sum()) > num_samples_umi
print(("{} of {} variants will remain after the UMI filter").format(
    umi_filter.sum(), freq.shape[1]))

52 of 133 variants will remain after the UMI filter


### Keep variants that were targeted
In most projects there are a number of variants that we would like to report, even if they are not seen in the sample set. We would like to stop those variants from being removed by the above filters.

In [16]:
# RUN
targ = freq.columns.get_level_values("Targeted") == "Yes"

### Combine filters
Keep the variants that are either targeted or passing filters

In [17]:
variant_mask = targ | (wsaf_filter & umi_filter)
print(("{} variants will remain in the final call set.\n"
       "{} variants were targeted and will be kept; and {} will be removed by "
       "the combined UMI and WSAF filters.").format(
    variant_mask.sum(), targ.sum(), (wsaf_filter & umi_filter).sum()))

79 variants will remain in the final call set.
60 variants were targeted and will be kept; and 32 will be removed by the combined UMI and WSAF filters.


## Filter data tables with the combined filters

In [18]:
filtered_genotypes = genotypes.loc[:, variant_mask]
filtered_genotypes.to_csv(os.path.join(wdir, "final_filtered_genotypes.csv"))
filtered_genotypes.head()

Gene ID              PF3D7_0417200                                    \
Gene                       dhfr-ts                                     
Mutation Name     dhfr-ts-Ala16Val dhfr-ts-Asn51Ile dhfr-ts-Cys59Arg   
ExonicFunc        missense_variant missense_variant missense_variant   
AA Change                 Ala16Val         Asn51Ile         Cys59Arg   
Targeted                       Yes              Yes              Yes   
KB-04-06-PRX-04-1              NaN              NaN              NaN   
AG-00-45-PRX-00-1              0.0              2.0              2.0   
KB-07-13-PRX-07-1              0.0              2.0              2.0   
AM-04-08-PRX-04-1              0.0              2.0              2.0   
AM-07-84-PRX-07-1              0.0              2.0              2.0   

Gene ID                                                                  \
Gene                                                                      
Mutation Name     dhfr-ts-Ile164Leu dhfr-ts-Ser108Asn dhfr-ts-Ser108Thr   
ExonicFunc         missense_variant  missense_variant  missense_variant   
AA Change                 Ile164Leu         Ser108Asn         Ser108Thr   
Targeted                        Yes               Yes               Yes   
KB-04-06-PRX-04-1               NaN               NaN               NaN   
AG-00-45-PRX-00-1               NaN               2.0               0.0   
KB-07-13-PRX-07-1               2.0               2.0               0.0   
AM-04-08-PRX-04-1               1.0               2.0               0.0   
AM-07-84-PRX-07-1               2.0               2.0               0.0   

Gene ID              PF3D7_0523000                   \
Gene                          mdr1                    
Mutation Name      mdr1-Asn1042Asp   mdr1-Asn652Asp   
ExonicFunc        missense_variant missense_variant   
AA Change               Asn1042Asp        Asn652Asp   
Targeted                       Yes               No   
KB-04-06-PRX-04-1              NaN              NaN   
AG-00-45-PRX-00-1              0.0              0.0   
KB-07-13-PRX-07-1              0.0              0.0   
AM-04-08-PRX-04-1              0.0              NaN   
AM-07-84-PRX-07-1              0.0              0.0   

Gene ID                                                                    \
Gene                                                                        
Mutation Name           mdr1-Asn659_Asn661del       mdr1-Asn660_Asn661del   
ExonicFunc        disruptive_inframe_deletion disruptive_inframe_deletion   
AA Change                    Asn659_Asn661del            Asn660_Asn661del   
Targeted                                   No                          No   
KB-04-06-PRX-04-1                         NaN                         NaN   
AG-00-45-PRX-00-1                         0.0                         0.0   
KB-07-13-PRX-07-1                         0.0                         0.0   
AM-04-08-PRX-04-1                         NaN                         NaN   
AM-07-84-PRX-07-1                         0.0                         0.0   

Gene ID            ...    PF3D7_1343700                                    \
Gene               ...              k13                                     
Mutation Name      ...    k13-Met476Ile    k13-Phe446Ile    k13-Pro441Leu   
ExonicFunc         ... missense_variant missense_variant missense_variant   
AA Change          ...        Met476Ile        Phe446Ile        Pro441Leu   
Targeted           ...              Yes              Yes              Yes   
KB-04-06-PRX-04-1  ...              0.0              NaN              NaN   
AG-00-45-PRX-00-1  ...              0.0              0.0              0.0   
KB-07-13-PRX-07-1  ...              0.0              0.0              0.0   
AM-04-08-PRX-04-1  ...              0.0              0.0              0.0   
AM-07-84-PRX-07-1  ...              0.0              0.0              0.0   

Gene ID                                                               \
Gene   

In [19]:
filtered_prevalences = prevalences.loc[:, variant_mask]
filtered_prevalences.to_csv(os.path.join(wdir, "final_filtered_prevalences_input_table.csv"))
filtered_prevalences.head()

Gene ID              PF3D7_0417200                                    \
Gene                       dhfr-ts                                     
Mutation Name     dhfr-ts-Ala16Val dhfr-ts-Asn51Ile dhfr-ts-Cys59Arg   
ExonicFunc        missense_variant missense_variant missense_variant   
AA Change                 Ala16Val         Asn51Ile         Cys59Arg   
Targeted                       Yes              Yes              Yes   
KB-04-06-PRX-04-1              NaN              NaN              NaN   
AG-00-45-PRX-00-1              0.0              1.0              1.0   
KB-07-13-PRX-07-1              0.0              1.0              1.0   
AM-04-08-PRX-04-1              0.0              1.0              1.0   
AM-07-84-PRX-07-1              0.0              1.0              1.0   

Gene ID                                                                  \
Gene                                                                      
Mutation Name     dhfr-ts-Ile164Leu dhfr-ts-Ser108Asn dhfr-ts-Ser108Thr   
ExonicFunc         missense_variant  missense_variant  missense_variant   
AA Change                 Ile164Leu         Ser108Asn         Ser108Thr   
Targeted                        Yes               Yes               Yes   
KB-04-06-PRX-04-1               NaN               NaN               NaN   
AG-00-45-PRX-00-1               NaN               1.0               0.0   
KB-07-13-PRX-07-1               1.0               1.0               0.0   
AM-04-08-PRX-04-1               1.0               1.0               0.0   
AM-07-84-PRX-07-1               1.0               1.0               0.0   

Gene ID              PF3D7_0523000                   \
Gene                          mdr1                    
Mutation Name      mdr1-Asn1042Asp   mdr1-Asn652Asp   
ExonicFunc        missense_variant missense_variant   
AA Change               Asn1042Asp        Asn652Asp   
Targeted                       Yes               No   
KB-04-06-PRX-04-1              NaN              NaN   
AG-00-45-PRX-00-1              0.0              0.0   
KB-07-13-PRX-07-1              0.0              0.0   
AM-04-08-PRX-04-1              0.0              NaN   
AM-07-84-PRX-07-1              0.0              0.0   

Gene ID                                                                    \
Gene                                                                        
Mutation Name           mdr1-Asn659_Asn661del       mdr1-Asn660_Asn661del   
ExonicFunc        disruptive_inframe_deletion disruptive_inframe_deletion   
AA Change                    Asn659_Asn661del            Asn660_Asn661del   
Targeted                                   No                          No   
KB-04-06-PRX-04-1                         NaN                         NaN   
AG-00-45-PRX-00-1                         0.0                         0.0   
KB-07-13-PRX-07-1                         0.0                         0.0   
AM-04-08-PRX-04-1                         NaN                         NaN   
AM-07-84-PRX-07-1                         0.0                         0.0   

Gene ID            ...    PF3D7_1343700                                    \
Gene               ...              k13                                     
Mutation Name      ...    k13-Met476Ile    k13-Phe446Ile    k13-Pro441Leu   
ExonicFunc         ... missense_variant missense_variant missense_variant   
AA Change          ...        Met476Ile        Phe446Ile        Pro441Leu   
Targeted           ...              Yes              Yes              Yes   
KB-04-06-PRX-04-1  ...              0.0              NaN              NaN   
AG-00-45-PRX-00-1  ...              0.0              0.0              0.0   
KB-07-13-PRX-07-1  ...              0.0              0.0              0.0   
AM-04-08-PRX-04-1  ...              0.0              0.0              0.0   
AM-07-84-PRX-07-1  ...              0.0              0.0              0.0   

Gene ID                                                               \
Gene   

# Calculate Prevalence in Each Region

In [20]:
metadata_file = '/opt/prevalence_metadata/PRX-00_metadata.csv'
prevalences_input_table = 'prevalences_input_table.csv'
UMI_suffix = '-PRX-00-1'
output_summary_table = 'prevalence_summary.tsv'

In [21]:
mutations = [
"crt-Cys72Ser",
"crt-Val73Leu",
"crt-Met74Ile",
"crt-Asn75Glu",
"crt-Lys76Thr",
"crt-Asn326Ser",
"crt-Ile356Thr",
"dhfr-ts-Asn51Ile",
"dhfr-ts-Cys59Arg",
"dhfr-ts-Ser108Asn",
"dhfr-ts-Ser108Thr",
"dhfr-ts-Ile164Leu",
"dhps-Ala437Gly",
"dhps-Ala581Gly",
"dhps-Ala613Ser",
"dhps-Ala613Thr",
"dhps-Ile431Val",
"dhps-Lys540Glu",
"dhps-Ser436Ala",
"dhps-Ser436Phe",
"k13-Ala675Val",
"k13-Arg539Thr",
"k13-Arg561His",
"k13-Arg622Ile",
"k13-Cys469Phe",
"k13-Cys469Tyr",
"k13-Cys580Tyr",
"k13-Pro441Leu",
"k13-Tyr493His",
"k13-Val568Gly",
"mdr1-Asn1042Asp",
"mdr1-Asn86Phe",
"mdr1-Asn86Tyr",
"mdr1-Asp1246Tyr",
"mdr1-Ser1034Cys",
"mdr1-Tyr184Phe"]


In [22]:
cap.calculate_prevalences(metadata_file,
                          prevalences_input_table,
                          UMI_suffix,
                          mutations,
                          output_summary_table)

prevalences_output = pd.read_csv(output_summary_table,sep='\t', index_col=0)
prevalences_output.head()

,dhfr-ts-Asn51Ile,dhfr-ts-Cys59Arg,dhfr-ts-Ile164Leu,dhfr-ts-Ser108Asn,dhfr-ts-Ser108Thr,mdr1-Asn1042Asp,mdr1-Asn86Tyr,mdr1-Asp1246Tyr,mdr1-Ser1034Cys,mdr1-Tyr184Phe,...,k13-Ala675Val,k13-Arg539Thr,k13-Arg561His,k13-Arg622Ile,k13-Cys469Phe,k13-Cys469Tyr,k13-Cys580Tyr,k13-Pro441Leu,k13-Tyr493His,k13-Val568Gly
Sites,,,,,,,,,,,,,,,,,,,,,
Agago,1.0 (18/18),0.7777777777777778 (14/18),0.07142857142857142 (1/14),1.0 (16/16),0 (0/16),0 (0/20),0 (0/18),0.14285714285714285 (2/14),0 (0/20),0.65 (13/20),...,0 (0/20),0 (0/19),0 (0/15),0 (0/19),0 (0/19),0 (0/19),0 (0/19),0 (0/19),0 (0/16),0 (0/15)
Amolatar,1.0 (13/13),1.0 (13/13),0 (0/8),1.0 (8/8),0 (0/8),0 (0/18),0 (0/14),0 (0/6),0 (0/18),1.0 (14/14),...,0 (0/18),0 (0/18),0 (0/9),0 (0/18),0 (0/16),0 (0/16),0 (0/18),0 (0/16),0 (0/14),0 (0/9)
Kabale,1.0 (19/19),1.0 (19/19),0.23529411764705882 (4/17),1.0 (16/16),0 (0/16),0 (0/19),0 (0/19),0 (0/16),0 (0/19),0.42105263157894735 (8/19),...,0 (0/19),0 (0/19),0 (0/17),0 (0/20),0.21052631578947367 (4/19),0 (0/19),0 (0/20),0 (0/19),0 (0/15),0 (0/17)
Kole,1.0 (11/11),0.9090909090909091 (10/11),0 (0/7),1.0 (7/7),0 (0/7),0 (0/17),0 (0/15),0 (0/11),0 (0/17),0.6153846153846154 (8/13),...,0 (0/18),0 (0/19),0 (0/15),0 (0/18),0 (0/16),0 (0/16),0 (0/18),0 (0/16),0 (0/16),0 (0/15)
overall,1.0 (61/61),0.9180327868852459 (56/61),0.10869565217391304 (5/46),1.0 (47/47),0 (0/47),0 (0/74),0 (0/66),0.0425531914893617 (2/47),0 (0/74),0.6515151515151515 (43/66),...,0 (0/75),0 (0/75),0 (0/56),0 (0/75),0.05714285714285714 (4/70),0 (0/70),0 (0/75),0 (0/70),0 (0/61),0 (0/56)


# Plot Choropleth

In [23]:
########### USER INPUT #################
region_of_interest = 'uganda'
w = widgets.Dropdown(
    options=['uganda', 'tanzania'],
    value='uganda',
    description='region:',
    disabled=False,
)
display(w)

Dropdown(description='region:', options=('uganda', 'tanzania'), value='uganda')

In [24]:
#RUN
with urlopen('https://raw.githubusercontent.com/simkin-bioinformatics/geojson_files/main/uganda-with-regions.geojson') as response:
    uganda_regions = json.load(response)
with urlopen('https://raw.githubusercontent.com/simkin-bioinformatics/geojson_files/main/tanzania-with-regions.geojson') as response:
    tanzania_regions = json.load(response)


# load the inputs
region_dict = {'uganda':[uganda_regions,5.8, {"lat": 1.3733, "lon": 32.2903}],
               'tanzania':[tanzania_regions,4, {"lat": -6.3690, "lon": 34.8888}]}

prevalence_summary_df = pd.read_csv(output_summary_table, sep='\t')

# identify columns that are variants not headers
variant_columns=prevalence_summary_df.columns.difference(['Sites'])

# get a list of every variant from the dataframe and convert prevalence values to floats
variant_list = []
for column in variant_columns:
    prevalence_summary_df[column]=[float(x.split()[0]) for x in prevalence_summary_df[column]]
    variant_list.append(column)

def display_choropleth(variant):
    json_file = region_dict[w.value][0]
    fig = px.choropleth_mapbox(prevalence_summary_df, 
                                geojson=json_file, 
                                locations='Sites', 
                                color=variant,
                                color_continuous_scale="reds",
                                mapbox_style="open-street-map",
                                featureidkey="properties.name",
                                zoom=region_dict[w.value][1], 
                                center = region_dict[w.value][2],
                                labels={'Sites':'Site'},
                                range_color=(0,1),
    )
    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    fig.update_layout(height=500)
    return fig

interact(display_choropleth, variant=variant_list);

interactive(children=(Dropdown(description='variant', options=('crt-Asn326Ser', 'crt-Asn75Glu', 'crt-Cys72Ser'…

In [25]:
!jupyter nbconvert --to html prevalence_plotting.ipynb

[NbConvertApp] Converting notebook prevalence_plotting.ipynb to html
[NbConvertApp] Writing 730157 bytes to prevalence_plotting.html
